In [1]:
from transformers import AutoConfig, AutoModelForCausalLM
# from transformers import MistralForCausalLM
# from fast_llm.models.ssm.external.apriel_15b_hybrid.configuration_ssm_hybrid_apriel15b import AprielSSMHybridConfig
# from fast_llm.models.ssm.external.apriel_15b_hybrid.modeling_ssm_hybrid_apriel15b import AprielSSMHybridForCausalLM
# autoreload changes to the code
%reload_ext autoreload
%autoreload 2


/home/toolkit/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model_path = "/mnt/checkpoints/fast_llm_exp/slam_ssm_distill/15bch-ifrhyb20l32h-bs768-lr0.0003-lrs0-0-0-0-sl4096_ti1000_lm2/export/apriel_ssm_thinker_hybrid/1000"
# AutoConfig.from_pretrained(model_path, trust_remote_code=True)

In [3]:
# model_path = "/mnt/checkpoints/fast_llm_exp/slam_ssm_distill/15bch-ifrhyb20l32h-bs768-lr0.0003-lrs0-0-0-0-sl4096_ti1000_lm2/export/apriel_ssm_thinker_hybrid/1000"
# m = AutoModelForCausalLM.from_pretrained(
#     model_path, trust_remote_code=True,
#     config=AutoConfig.from_pretrained(model_path, trust_remote_code=True),
# )

### Slam 15B upcycled

 Lead the weights of https://huggingface.co/ServiceNow-AI/Slam-15B-Upcycled/ into Thiked modeling, it shoudl work

In [2]:
import sys
sys.path.append("/home/toolkit/dev/fml-ops/__oo_playground")
from results_analysis.results_loader import ResultsLoader
layer_importance_path = "/mnt/evaluations/training_evaluation/model_runs/lm_eval_runner/apriel_ssm_importance/"
results_loader = ResultsLoader(layer_importance_path)

results_loader.deserialize_results()
results_df = results_loader.to_df()
results_df["layer_index"] = results_df.apply(lambda row: int(row["model_name_sanitized"].split("_")[-1] if "layers_" in row["model_name_sanitized"] else -1), axis=1)
results_df = results_df[results_df["metric"] == "acc_norm"]
columns_to_keep = ["layer_index", "metric_value"]
results_df = results_df[columns_to_keep]
layer_importance = results_df.groupby("layer_index").mean()
layer_importance = layer_importance.sort_values(by="metric_value", ascending=False).reset_index()
layer_importance = layer_importance[layer_importance["layer_index"]!= -1]
layer_importance = list(layer_importance["layer_index"])

In [3]:
layer_importance

[22,
 25,
 20,
 31,
 29,
 46,
 23,
 26,
 33,
 24,
 47,
 27,
 21,
 41,
 17,
 18,
 34,
 42,
 44,
 30,
 16,
 8,
 43,
 35,
 19,
 38,
 15,
 28,
 32,
 45,
 37,
 40,
 7,
 36,
 13,
 10,
 5,
 39,
 6,
 14,
 4,
 12,
 9,
 48,
 1,
 3,
 11,
 49,
 0]

In [4]:
path_thinker = "/mnt/checkpoints/upstream/Apriel-Nemotron-15b-Thinker"
n_ssm = 25

config_thinker = AutoConfig.from_pretrained(path_thinker)
hybrid_block_layout = ["t"] * config_thinker.num_hidden_layers

for i in range(n_ssm):
    hybrid_block_layout[layer_importance[i]] = "m2d"

config_hybrid = AprielSSMHybridConfig(
    **config_thinker.to_dict(),
    hybrid_block_layout=hybrid_block_layout,
    ssm_cfg = {
            "d_state": 64,
            "n_v_heads": 32,
            "n_qk_heads": 32,
            "expand": 1,
            "chunk_size": 128,
            "activation": "identity",
            "bias": False,
            "d_conv": 4,
            "d_inner": 32 * 128
        }
)
model_hybrid = AprielSSMHybridForCausalLM(config_hybrid)


In [5]:
hybrid_block_layout

['t',
 't',
 't',
 't',
 't',
 't',
 't',
 't',
 'm2d',
 't',
 't',
 't',
 't',
 't',
 't',
 't',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 't',
 'm2d',
 'm2d',
 'm2d',
 't',
 'm2d',
 'm2d',
 'm2d',
 't',
 't',
 't',
 't',
 't',
 'm2d',
 'm2d',
 'm2d',
 'm2d',
 't',
 'm2d',
 'm2d',
 't',
 't']

In [6]:
path_base = "/mnt/checkpoints/upstream/Slam-15B-Upcycled"
# path_base = "/mnt/checkpoints/upstream/Slam-15B-Upcycled-mistral"
model_base = MistralForCausalLM.from_pretrained(path_base)


You are using a model of type llama to instantiate a model of type mistral. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.52it/s]


In [7]:
model_hybrid.load_state_dict(model_base.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['model.layers.8.mixer.z_bias', 'model.layers.8.mixer.D', 'model.layers.8.mixer.in_proj.weight', 'model.layers.8.mixer.conv1d.weight', 'model.layers.8.mixer.conv1d.bias', 'model.layers.8.mixer.out_proj.weight', 'model.layers.16.mixer.z_bias', 'model.layers.16.mixer.D', 'model.layers.16.mixer.in_proj.weight', 'model.layers.16.mixer.conv1d.weight', 'model.layers.16.mixer.conv1d.bias', 'model.layers.16.mixer.out_proj.weight', 'model.layers.17.mixer.z_bias', 'model.layers.17.mixer.D', 'model.layers.17.mixer.in_proj.weight', 'model.layers.17.mixer.conv1d.weight', 'model.layers.17.mixer.conv1d.bias', 'model.layers.17.mixer.out_proj.weight', 'model.layers.18.mixer.z_bias', 'model.layers.18.mixer.D', 'model.layers.18.mixer.in_proj.weight', 'model.layers.18.mixer.conv1d.weight', 'model.layers.18.mixer.conv1d.bias', 'model.layers.18.mixer.out_proj.weight', 'model.layers.19.mixer.z_bias', 'model.layers.19.mixer.D', 'model.layers.19.mixer.in_proj.weight', 'model.laye

In [8]:
model_hybrid.save_pretrained("/mnt/checkpoints/ssm/apriel_ssm_base15b_hybrid_25ssm_leastimportant_32h_init_rand")


### Thinker 15B

In [1]:
import gc

import click
import torch
from transformers import AutoConfig, AutoModelForCausalLM
from transformers import MistralForCausalLM

from fast_llm.models.ssm.external.apriel_15b_hybrid.configuration_ssm_hybrid_apriel15b import AprielSSMHybridConfig
from fast_llm.models.ssm.external.apriel_15b_hybrid.modeling_ssm_hybrid_apriel15b import AprielSSMHybridForCausalLM

# enable file reload 
%load_ext autoreload
%autoreload 2

/home/toolkit/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Repalce specific layer from MOHAWK

In [3]:
l3checkpoint = "/mnt/checkpoints/ssm/apriel_ssm_thinker15b_hybrid_25ssm_leastimportant_32h_init_rand" #/mnt/checkpoints/ssm/iterative_hybrids_only_new_layer_train/apriel_ssm_thinker15b_hybrid_3ssm_leastimportant_32h_init_rand"
model_l3 = AprielSSMHybridForCausalLM.from_pretrained(l3checkpoint, device_map="cpu")
mohawk_checkpoint = "/mnt/checkpoints_fml/ssm/final_stitched_model_L0-49_resaved"
model_mohawk = AprielSSMHybridForCausalLM.from_pretrained(mohawk_checkpoint, device_map="cpu")




Loading checkpoint shards: 100%|██████████| 14/14 [00:00<00:00, 232.07it/s]


In [2]:
model_l3.config.hybrid_block_layout

NameError: name 'model_l3' is not defined

In [6]:

# layer_ids = "47"
sdm = model_mohawk.state_dict()
# layer_sd = {k: v for k, v in sdm.items() if f"layers.{layer_ids}" in k}.copy()
# r = model_l3.load_state_dict(layer_sd, strict=False)
r = model_l3.load_state_dict(sdm, strict=False)
print(r.unexpected_keys)

['model.layers.0.mixer.z_bias', 'model.layers.0.mixer.D', 'model.layers.0.mixer.in_proj.weight', 'model.layers.0.mixer.conv1d.weight', 'model.layers.0.mixer.conv1d.bias', 'model.layers.0.mixer.out_proj.weight', 'model.layers.1.mixer.z_bias', 'model.layers.1.mixer.D', 'model.layers.1.mixer.in_proj.weight', 'model.layers.1.mixer.conv1d.weight', 'model.layers.1.mixer.conv1d.bias', 'model.layers.1.mixer.out_proj.weight', 'model.layers.2.mixer.z_bias', 'model.layers.2.mixer.D', 'model.layers.2.mixer.in_proj.weight', 'model.layers.2.mixer.conv1d.weight', 'model.layers.2.mixer.conv1d.bias', 'model.layers.2.mixer.out_proj.weight', 'model.layers.3.mixer.z_bias', 'model.layers.3.mixer.D', 'model.layers.3.mixer.in_proj.weight', 'model.layers.3.mixer.conv1d.weight', 'model.layers.3.mixer.conv1d.bias', 'model.layers.3.mixer.out_proj.weight', 'model.layers.4.mixer.z_bias', 'model.layers.4.mixer.D', 'model.layers.4.mixer.in_proj.weight', 'model.layers.4.mixer.conv1d.weight', 'model.layers.4.mixer.con

In [7]:
# model_l3.save_pretrained("/mnt/checkpoints/ssm/iterative_hybrids_only_new_layer_train/apriel_ssm_thinker15b_hybrid_3ssm_leastimportant_32h_init_mohawk_layer24")
model_l3.save_pretrained("/mnt/checkpoints/ssm/apriel_ssm_thinker15b_hybrid_25ssm_leastimportant_32h_init_mohawk")

In [2]:
import sys
sys.path.append("/home/toolkit/dev/fml-ops/__oo_playground")
from results_analysis.results_loader import ResultsLoader
layer_importance_path = "/mnt/evaluations/training_evaluation/model_runs/lm_eval_runner/apriel_ssm_importance/"
results_loader = ResultsLoader(layer_importance_path)

results_loader.deserialize_results()
results_df = results_loader.to_df()
results_df["layer_index"] = results_df.apply(lambda row: int(row["model_name_sanitized"].split("_")[-1] if "layers_" in row["model_name_sanitized"] else -1), axis=1)
results_df = results_df[results_df["metric"] == "acc"]
columns_to_keep = ["layer_index", "metric_value"]
results_df = results_df[columns_to_keep]
layer_importance = results_df.groupby("layer_index").mean()
layer_importance = layer_importance.sort_values(by="metric_value", ascending=False).reset_index()
layer_importance = layer_importance[layer_importance["layer_index"]!= -1]
layer_importance = list(layer_importance["layer_index"])

In [3]:
layer_importance

[47,
 39,
 24,
 36,
 31,
 43,
 32,
 20,
 38,
 37,
 30,
 33,
 22,
 23,
 40,
 42,
 44,
 35,
 41,
 27,
 21,
 46,
 45,
 49,
 25,
 34,
 29,
 28,
 19,
 26,
 18,
 17,
 16,
 13,
 15,
 14,
 8,
 9,
 12,
 6,
 11,
 5,
 48,
 7,
 10,
 3,
 4,
 1,
 0]

In [5]:
path_thinker = "/mnt/checkpoints/upstream/Apriel-Nemotron-15b-Thinker"
n_ssm = 1

config_thinker = AutoConfig.from_pretrained(path_thinker)
config_thinker.num_hidden_layers = 5
hybrid_block_layout = ["t"] * config_thinker.num_hidden_layers
hybrid_block_layout[0] = "m2"
# hybrid_block_layout[0] = "m2d"

# for i in range(n_ssm):
#     # hybrid_block_layout[layer_importance[i]] = "m2d"
#     # if i % 2 == 0:
#         hybrid_block_layout[layer_importance[i]] = "m2"
        # hybrid_block_layout[i] = "m2"

# hybrid_block_layout = ["t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "t", "m2d", "t", "t", "t", "m2d", "t", "t", "t", "t", "t", "m2d", "m2d", "t", "t", "t", "t", "m2d", "m2d", "m2d", "m2d", "t", "t", "t", "m2d", "t", "t", "t", "m2d", "t", "m2d"]

config_hybrid = AprielSSMHybridConfig(
    **config_thinker.to_dict(),
    hybrid_block_layout=hybrid_block_layout,
    ssm_cfg = {
            "d_state": 64,
            "n_v_heads": 32,
            "n_qk_heads": 32,
            "expand": 1,
            "chunk_size": 128,
            "activation": "identity",
            "bias": False,
            "d_conv": 4,
            "d_inner": 32 * 128,
            # mamba2
            "expand": 2
        }
)
model_hybrid = AprielSSMHybridForCausalLM(config_hybrid)


In [6]:
hybrid_block_layout

['m2', 't', 't', 't', 't']

In [7]:
path_base = path_thinker
model_base = MistralForCausalLM.from_pretrained(path_base)

model_hybrid.load_state_dict(model_base.state_dict(), strict=False)


Loading checkpoint shards: 100%|██████████| 7/7 [00:18<00:00,  2.71s/it]


_IncompatibleKeys(missing_keys=['model.layers.0.mixer.A_log', 'model.layers.0.mixer.D', 'model.layers.0.mixer.conv1d.weight', 'model.layers.0.mixer.conv1d.bias', 'model.layers.0.mixer.in_proj.weight', 'model.layers.0.mixer.dt_proj.weight', 'model.layers.0.mixer.dt_proj.bias', 'model.layers.0.mixer.out_proj.weight'], unexpected_keys=['model.layers.5.self_attn.q_proj.weight', 'model.layers.5.self_attn.k_proj.weight', 'model.layers.5.self_attn.v_proj.weight', 'model.layers.5.self_attn.o_proj.weight', 'model.layers.5.mlp.gate_proj.weight', 'model.layers.5.mlp.up_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.5.input_layernorm.weight', 'model.layers.5.post_attention_layernorm.weight', 'model.layers.6.self_attn.q_proj.weight', 'model.layers.6.self_attn.k_proj.weight', 'model.layers.6.self_attn.v_proj.weight', 'model.layers.6.self_attn.o_proj.weight', 'model.layers.6.mlp.gate_proj.weight', 'model.layers.6.mlp.up_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.l

In [8]:
# model_hybrid.save_pretrained("/mnt/checkpoints/ssm/apriel_ssm_thinker15b_hybrid_1ssm_leastimportant_32h_init_rand") # 1 ssm
# model_hybrid.save_pretrained("/mnt/checkpoints/ssm/apriel_ssm_thinker15b_hybrid_1ssm_0th_32h_init_rand") # 1 ssm
# model_hybrid.save_pretrained("/mnt/checkpoints/ssm/apriel_ssm_thinker15b_hybrid_interleaved_ssm_starting0th_32h_init_rand") # 1 ssm
# model_hybrid.save_pretrained("/mnt/checkpoints/ssm/apriel_ssm_thinker15b_hybrid_1ssm_leastimportant_32h_init_rand") # 1 ssm
model_hybrid.save_pretrained("/mnt/checkpoints/ssm/apriel_ssm_thinker5lb_hybrid_1ssm_m2debug") # 1 ssm








In [3]:
path = "/mnt/checkpoints/ssm/apriel_ssm_thinker15b_hybrid_25ssm_leastimportant_32h_init_rand"
model = AprielSSMHybridForCausalLM.from_pretrained(path)

Loading checkpoint shards: 100%|██████████| 7/7 [00:40<00:00,  5.77s/it]


: 

In [ ]:
path_base = "/mnt/checkpoints/upstream/Apriel-Nemotron-15b-Thinker"
model_base = MistralForCausalLM.from_pretrained(path_base)

Loading checkpoint shards:  29%|██▊       | 2/7 [00:11<00:27,  5.53s/it]